# FineRec preprocess 1 Amazon

In [2]:
import gzip
from collections import defaultdict
from datetime import datetime
import os
import time
import pickle
import pandas as pd
import json as json
import numpy as np

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

meta_dataset = 'Sports_and_Outdoors'

interaction_path = './originalData/reviews_' + meta_dataset + '_5.json.gz'
# df = getDF('originalData/Tools_and_Home_Improvement.json.gz')
df_inter = getDF(interaction_path)


# userID-itemID-timestamp-reviews
df_inter = df_inter[['reviewerID', 'asin', 'unixReviewTime', 'reviewText']]

# toy test, 100,000 reviews
# df_inter = df_inter.head(6000)

def merge_user_item(user, item):
    return str(user)+str(item)

def words_counter(review):
    return len(str(review).split(' '))

def reduce_words(review):
    return str(review)[:500]

df_inter['useritem'] = df_inter.apply(lambda row: merge_user_item(row['reviewerID'], row['asin']), axis=1)
interaction = df_inter.rename(columns={'unixReviewTime':'review_time'})

# delete interactions with invalid reviews, like only one word in review
interaction['word_num'] = interaction.apply(lambda row: words_counter(row['reviewText']), axis=1)
interaction = interaction[interaction['word_num']>1]

# interaction: there maight be reapeated items in sequences
interaction = interaction[['useritem', 'reviewerID', 'asin', 'review_time', 'reviewText']]

# only retaining 500 characters in each review
interaction['re_review'] = interaction.apply(lambda row: reduce_words(row['reviewText']), axis=1)
interaction = interaction[['useritem', 'reviewerID', 'asin', 'review_time', 're_review']]
interaction = interaction.reset_index().rename(columns={'re_review':'reviewText'})
interaction = interaction[['useritem', 'reviewerID', 'asin', 'review_time', 'reviewText']]
interaction

,useritem,reviewerID,asin,review_time,reviewText
0,AIXZKN4ACSKI1881509818,AIXZKN4ACSKI,1881509818,1390694400,This came in on time and I am veru happy with ...
1,A1L5P841VIO02V1881509818,A1L5P841VIO02V,1881509818,1328140800,I had a factory Glock tool that I was using fo...
2,AB2W04NI4OEAD1881509818,AB2W04NI4OEAD,1881509818,1330387200,If you don't have a 3/32 punch or would like t...
3,A148SVSWKTJKU61881509818,A148SVSWKTJKU6,1881509818,1328400000,This works no better than any 3/32 punch you w...
4,AAAWJ6LW9WMOO1881509818,AAAWJ6LW9WMOO,1881509818,1366675200,I purchased this thinking maybe I need a speci...
5,A2XX2A4OJCDNLZ1881509818,A2XX2A4OJCDNLZ,1881509818,1351814400,"Needed this tool to really break down my G22, ..."
6,A283UOBQRUNM4Q1881509818,A283UOBQRUNM4Q,1881509818,1402358400,If u don't have it .. Get it. All you need to ...
7,AWG3H90WVZ0Z12094869245,AWG3H90WVZ0Z1,2094869245,1377907200,This light will no doubt capture the attention...
8,A3V52OTJHKIJZX2094869245,A3V52OTJHKIJZX,2094869245,1369612800,"Light and laser torch work well, very bright. ..."
9,A3SZBE5F3UQ9EC2094869245,A3SZBE5F3UQ9EC,2094869245,1383350400,Does everything it says it will do. I would li...


In [3]:
user_item1 = interaction
user_item2 = user_item1.dropna(axis=0)

item_count = pd.DataFrame(user_item2.groupby(user_item2['asin']).count())
item_num=item_count.reset_index()[['reviewerID','asin']].rename(columns={'reviewerID':'item_num'})
user_item3 = pd.merge(user_item2, item_num, how='left', on = 'asin')
user_item3 = user_item3[user_item3['item_num']>4]


user_item3.sort_values(by=["reviewerID","review_time"],inplace=True,ascending=[True,True])
user_click_num = pd.DataFrame(user_item3.groupby(user_item3['reviewerID']).count())
click_num=user_click_num.reset_index()[['reviewerID','asin']].rename(columns={'asin':'click_num'})
item_data6 = pd.merge(user_item3, click_num, how='left', on = 'reviewerID')

item_data7 = item_data6[item_data6['click_num']>4]
data = item_data7[['useritem', 'reviewerID', 'asin', 'review_time', 'reviewText']]

#  there is only one review for each user-item pair
nonrep_interaction = data.drop_duplicates(subset=['useritem'],keep='first',inplace=False)
data_review = nonrep_interaction[['useritem', 'reviewText']]
data_inter = data[['useritem', 'reviewerID', 'asin', 'review_time', 'reviewText']]

save_pre_path = './dict/' + meta_dataset
if not os.path.exists(save_pre_path):
    os.makedirs(save_pre_path)
save_path_review = save_pre_path + '/useritem_review.csv'
save_path_inter = save_pre_path + '/interaction.csv'
data_review.to_csv(save_path_review)
data_inter.to_csv(save_path_inter)
print('task solved!')

task solved!


### 处理 review -> ao

#  Attribute&opinion dict 处理成int，并形成 embedding table

### 合成数据

In [1]:
import os
import pickle
import time
import pandas as pd
import numpy as np



# synthic data, this should be obtained from pycharm processing 

import random
meta_dataset = 'Beauty'
inter_path = './dict/' + meta_dataset + '/interaction.csv'
interaction = pd.read_csv(inter_path)
interaction = interaction.reset_index()[['useritem', 'reviewerID', 'asin', 'review_time']]

ao_brand = interaction[['useritem']].head(20000)
ao_price = interaction[['useritem']].head(20000)
def syn_ao():
    return random.randint(0,1000)

ao_brand['brand'] = ao_brand.apply(lambda row: syn_ao(), axis=1)
ao_price['price'] = ao_price.apply(lambda row: syn_ao(), axis=1)
# pycharm 应该处理成这个格式

# to learn attribute embeddings
attri_dict = {}
attri_list = ['Brand', 'Ingredients', 'Effectiveness', 'Scent', 'Price', 'Color', 'Size']
for attri_t,a_idx in zip(attri_list, range(1,len(attri_list)+1)):
    attri_dict[attri_t] = a_idx

# pycharm should return data with the following format
ao_brand       

,useritem,brand
0,A00414041RD0BXM6WK0GXB007IY97U0,57
1,A00414041RD0BXM6WK0GXB00870XLDS,97
2,A00414041RD0BXM6WK0GXB008MIRO88,487
3,A00414041RD0BXM6WK0GXB00BQYYMN0,779
4,A00414041RD0BXM6WK0GXB00GRTQBTM,372
...,...,...
19995,A1ELD8EANACWMB00IP42FBA,125
19996,A1ELD8EANACWMB00C8YCFU4,159
19997,A1ELD8EANACWMB00KAL5JAU,95
19998,A1ELD8EANACWMB00KHH2VOY,398


## 读取 attribute-opinion mat

In [1]:
import os
import pickle
import time
import pandas as pd
import numpy as np

meta_dataset = 'Sports_and_Outdoors'
# interaction data

if meta_dataset == 'Beauty' or meta_dataset == 'Beauty_4':
    # 'Brand','Ingredients','Effectiveness','Scent','Price','Color','Size' ,'Compatibility', 'Longevity'
    attr_list = ['Brand','Ingredients','Effectiveness','Scent','Price','Color','Size']
elif meta_dataset == 'Cell_Phones_and_Accessories':
    # Brand, Performance, color, Battery ,Connectivity,Price, Size
    attr_list = ['Brand', 'Performance', 'Color', 'Battery' ,'Connectivity','Price', 'Size']
elif meta_dataset == 'Clothing_Shoes_and_Jewelry':
    # Price, Style, Size, Fabric, Brand, Quality, Color
    attr_list = ['Price', 'Style', 'Size', 'Fabric', 'Brand', 'Quality', 'Color']
elif meta_dataset == 'Home_and_Kitchen':
    # Price, Brand, Quality, Material, Size, Appearance, Washability, Functionality, Style, Durability,
    attr_list = ['Price', 'Brand', 'Quality', 'Material', 'Size', 'Appearance', 'Washability', 'Functionality', 'Style']
elif meta_dataset == 'Sports_and_Outdoors':
    # Price, Quality, Functionality,Size,Protection,Material,Comfort
    attr_list = ['Price', 'Quality', 'Functionality', 'Size', 'Protection', 'Material', 'Comfort']

inter_path = './dict/' + meta_dataset + '/interaction.csv'
attr_pre_path = './dict/' + meta_dataset +  '/attr_opin/'
attr_path_list = os.listdir(attr_pre_path)
attr_num = len(attr_path_list)
interaction = pd.read_csv(inter_path)
# interaction = interaction.head(21000)
interaction = interaction.reset_index()[['useritem', 'reviewerID', 'asin', 'review_time', 'reviewText']]
for x in attr_list:
    attr_path = attr_pre_path + 'ui_attr_' + str(x) + '.csv'
    ao = pd.read_csv(attr_path).reset_index()[['useritem', x]]
    interaction = pd.merge(interaction, ao, how='left', on = 'useritem')
interaction

,useritem,reviewerID,asin,review_time,reviewText,Price,Quality,Functionality,Size,Protection,Material,Comfort
0,A00046902LP5YSDV0VVNFB0010O748Q,A00046902LP5YSDV0VVNF,B0010O748Q,1353628800,i tested this out and like most wont leave hom...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A00046902LP5YSDV0VVNFB00178CS4K,A00046902LP5YSDV0VVNF,B00178CS4K,1353628800,for cheaper item this was worth the price. it ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A00046902LP5YSDV0VVNFB001THHXA8,A00046902LP5YSDV0VVNF,B001THHXA8,1353628800,i would say buy this for most outdoorsman use....,33698.0,NaN,NaN,33698.0,NaN,NaN,NaN
3,A00046902LP5YSDV0VVNFB004U8CP88,A00046902LP5YSDV0VVNF,B004U8CP88,1353628800,no way to attach a canister of fuel from u.s.a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A00046902LP5YSDV0VVNFB007BO931U,A00046902LP5YSDV0VVNF,B007BO931U,1353628800,"good purchase, this knife used with lansky sys...",33699.0,33699.0,175225.0,175225.0,175225.0,33699.0,355447.0
5,A0029274J35Q1MYNKUWOB0042X0TUM,A0029274J35Q1MYNKUWO,B0042X0TUM,1361404800,Purchased this shooting bench rest to use to b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,A0029274J35Q1MYNKUWOB0010O748Q,A0029274J35Q1MYNKUWO,B0010O748Q,1362096000,"Everyone should have an item like this ""Emerge...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A0029274J35Q1MYNKUWOB000BPEDWK,A0029274J35Q1MYNKUWO,B000BPEDWK,1364947200,The Seller did properly describe the trap set ...,33700.0,116953.0,175226.0,230951.0,266521.0,303123.0,NaN
8,A0029274J35Q1MYNKUWOB003DWM6V8,A0029274J35Q1MYNKUWO,B003DWM6V8,1392249600,"Nice fiber optic sight, and it produces a red/...",33701.0,33701.0,33701.0,33701.0,33701.0,33701.0,334954.0
9,A0029274J35Q1MYNKUWOB004W2KRTG,A0029274J35Q1MYNKUWO,B004W2KRTG,1392854400,"This ammo case is nice, well thought out, I'm ...",NaN,NaN,175227.0,NaN,NaN,NaN,NaN


In [2]:
if meta_dataset == 'Home_and_Kitchen':
    interaction = interaction[['reviewerID', 'asin', 'review_time', 'reviewText', attr_list[0], attr_list[1], attr_list[2], attr_list[3], attr_list[4], attr_list[5], attr_list[6], attr_list[7], attr_list[8]]]
else:
    interaction = interaction[['reviewerID', 'asin', 'review_time', 'reviewText', attr_list[0], attr_list[1], attr_list[2], attr_list[3], attr_list[4], attr_list[5], attr_list[6]]]
def set_zero(attr):
    if attr == 'NaN':
        results = 0
    else:
        results = int(attr)
    return results
    
user_item = interaction.fillna(0)

def check_zero(attr1, attr2, attr3, attr4, attr5, attr6, attr7, attr8=0, attr9=0):
    light = attr1 + attr2 + attr3 + attr4 + attr5 + attr6 + attr7 +attr8+attr9
    return light
if meta_dataset == 'Home_and_Kitchen':
    user_item['light'] = user_item.apply(lambda row: check_zero(row[attr_list[0]], row[attr_list[1]],row[attr_list[2]],row[attr_list[3]],row[attr_list[4]],row[attr_list[5]],row[attr_list[6]], row[attr_list[7]], row[attr_list[8]]), axis=1)

else:
    user_item['light'] = user_item.apply(lambda row: check_zero(row[attr_list[0]], row[attr_list[1]],row[attr_list[2]],row[attr_list[3]],row[attr_list[4]],row[attr_list[5]],row[attr_list[6]]), axis=1)
user_item = user_item[user_item['light'] != 0]
if meta_dataset == 'Home_and_Kitchen':
    user_item = user_item[['reviewerID','asin', 'review_time', 'reviewText', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6],attr_list[7],attr_list[8]]]
else:
    user_item = user_item[['reviewerID','asin', 'review_time', 'reviewText', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]

# 5-cores
item_count = pd.DataFrame(user_item.groupby(user_item['asin']).count())
item_num = item_count.reset_index()[['reviewerID','asin']].rename(columns={'reviewerID':'item_num'})
data1 = pd.merge(user_item, item_num, how='left', on = 'asin')
data2 = data1[data1['item_num']>4]

data2.sort_values(by=["reviewerID","review_time"],inplace=True,ascending=[True,True])
user_click_num = pd.DataFrame(data2.groupby(data2['reviewerID']).count())
click_num=user_click_num.reset_index()[['reviewerID','asin']].rename(columns={'asin':'click_num'})
data3 = pd.merge(data2, click_num, how='left', on = 'reviewerID')
# 删除长度小于5的session
data4 = data3[data3['click_num']>4]

if meta_dataset == 'Home_and_Kitchen':
    data = data4[['reviewerID', 'asin', 'review_time', 'reviewText', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6], attr_list[7], attr_list[8]]]
else:
    data = data4[['reviewerID', 'asin', 'review_time', 'reviewText', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]
data

/home/xbz5319/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,reviewerID,asin,review_time,reviewText,Price,Quality,Functionality,Size,Protection,Material,Comfort
2,A0029274J35Q1MYNKUWO,B000BPEDWK,1364947200,The Seller did properly describe the trap set ...,33700.0,116953.0,175226.0,230951.0,266521.0,303123.0,0.0
3,A0029274J35Q1MYNKUWO,B003DWM6V8,1392249600,"Nice fiber optic sight, and it produces a red/...",33701.0,33701.0,33701.0,33701.0,33701.0,33701.0,334954.0
4,A0029274J35Q1MYNKUWO,B004W2KRTG,1392854400,"This ammo case is nice, well thought out, I'm ...",0.0,0.0,175227.0,0.0,0.0,0.0,0.0
5,A0029274J35Q1MYNKUWO,B008HSROPG,1393200000,For the money this belt is probably a good buy...,33702.0,116954.0,153038.0,230952.0,0.0,116954.0,0.0
6,A0029274J35Q1MYNKUWO,B004W2KRP0,1393804800,"I really like this 12 ga shotgun ammo pouch, i...",33701.0,33701.0,0.0,0.0,0.0,0.0,0.0
7,A0029274J35Q1MYNKUWO,B009V5HXGO,1397001600,"This Is A Very Neat Tactical Vest, &#34;A Best...",33703.0,116955.0,175228.0,230953.0,266522.0,303124.0,230953.0
8,A0029274J35Q1MYNKUWO,B0002IOARE,1402876800,"It does ""Work"" as stated, and I would have giv...",0.0,0.0,175229.0,0.0,0.0,0.0,0.0
9,A0029274J35Q1MYNKUWO,B001OLU4UA,1402876800,"This is a very nice case, it's size is (34x10)...",33704.0,116956.0,0.0,230954.0,266523.0,33704.0,0.0
10,A0029274J35Q1MYNKUWO,B007IWZMA6,1402876800,"Nice Sling, it seems to be heavy duty, and I w...",33705.0,0.0,0.0,230955.0,0.0,33705.0,0.0
15,A005011233SVRED9Q0VY0,B0027I0D4G,1355788800,"This is a really well made knife, for such a l...",0.0,0.0,175231.0,0.0,0.0,0.0,0.0


In [3]:
# 获得item data feature for baseline
item_data_path = './dict/' + meta_dataset + '/itemdata.csv'
item_data = pd.read_csv(item_data_path)
data = pd.merge(data, item_data, how='left', on = 'asin')
if  meta_dataset == 'Home_and_Kitchen':
    data = data[['reviewerID', 'asin', 'title', 'cate','review_time', 'reviewText', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6], attr_list[7], attr_list[8]]]
else:
    data = data[['reviewerID', 'asin', 'title', 'cate','review_time', 'reviewText', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]
def reset_extra_fea(feature):
    results = str(feature)
    if results == 'nan':
        resluts = 'unknown'
    return results
data['text']=data.title.map(reset_extra_fea)
data['category']=data.cate.map(reset_extra_fea)
if meta_dataset == 'Home_and_Kitchen':
    data = data[['reviewerID', 'asin', 'review_time', 'reviewText','text', 'category', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6], attr_list[7], attr_list[8]]]
else:
    data = data[['reviewerID', 'asin', 'review_time', 'reviewText','text', 'category', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]
data

,reviewerID,asin,review_time,reviewText,text,category,Price,Quality,Functionality,Size,Protection,Material,Comfort
0,A0029274J35Q1MYNKUWO,B000BPEDWK,1364947200,The Seller did properly describe the trap set ...,Duke Set Tool for #220 / #330 Body Traps,Hunting,33700.0,116953.0,175226.0,230951.0,266521.0,303123.0,0.0
1,A0029274J35Q1MYNKUWO,B003DWM6V8,1392249600,"Nice fiber optic sight, and it produces a red/...",nan,nan,33701.0,33701.0,33701.0,33701.0,33701.0,33701.0,334954.0
2,A0029274J35Q1MYNKUWO,B004W2KRTG,1392854400,"This ammo case is nice, well thought out, I'm ...",nan,nan,0.0,0.0,175227.0,0.0,0.0,0.0,0.0
3,A0029274J35Q1MYNKUWO,B008HSROPG,1393200000,For the money this belt is probably a good buy...,nan,nan,33702.0,116954.0,153038.0,230952.0,0.0,116954.0,0.0
4,A0029274J35Q1MYNKUWO,B004W2KRP0,1393804800,"I really like this 12 ga shotgun ammo pouch, i...",nan,nan,33701.0,33701.0,0.0,0.0,0.0,0.0,0.0
5,A0029274J35Q1MYNKUWO,B009V5HXGO,1397001600,"This Is A Very Neat Tactical Vest, &#34;A Best...",nan,nan,33703.0,116955.0,175228.0,230953.0,266522.0,303124.0,230953.0
6,A0029274J35Q1MYNKUWO,B0002IOARE,1402876800,"It does ""Work"" as stated, and I would have giv...",nan,nan,0.0,0.0,175229.0,0.0,0.0,0.0,0.0
7,A0029274J35Q1MYNKUWO,B001OLU4UA,1402876800,"This is a very nice case, it's size is (34x10)...","VISM by NcStar Gun Case (CV2910-34), Black, 34...",Rifle Cases,33704.0,116956.0,0.0,230954.0,266523.0,33704.0,0.0
8,A0029274J35Q1MYNKUWO,B007IWZMA6,1402876800,"Nice Sling, it seems to be heavy duty, and I w...",nan,nan,33705.0,0.0,0.0,230955.0,0.0,33705.0,0.0
9,A005011233SVRED9Q0VY0,B0027I0D4G,1355788800,"This is a really well made knife, for such a l...",Fury Tactical Boot Knife,Tactical Knives,0.0,0.0,175231.0,0.0,0.0,0.0,0.0


In [4]:
# re-order item&session reviewer -> sessionID, asin -> itemID

# dict = {reviewerID: sessionID}
reviewerID2sessionID = {}
# dict = {asinID: itemID(renumber starting form 1)}
asin2itemID = {}

sessionNum = 0
itemNum = 0

# renumber the item & session 
for _, row in data.iterrows():
    if row['reviewerID'] not in reviewerID2sessionID:
        sessionNum += 1
        reviewerID2sessionID[row['reviewerID']] = sessionNum
    if row['asin'] not in asin2itemID:
        itemNum += 1
        asin2itemID[row['asin']] = itemNum
    
print('#session: ', sessionNum)
print('#item: ', itemNum)

def reSession(reviewerID):
    if reviewerID in reviewerID2sessionID:
        return reviewerID2sessionID[reviewerID]
    else:
        print('session is not recorded')
        return 'none'
    
def reItem(asin):
    if asin in asin2itemID:
        return asin2itemID[asin]
    else:
        print('item is not recorded')
        return 'none'

data['sessionID']=data.reviewerID.map(reSession)
data['itemID']=data.asin.map(reItem)
if meta_dataset == 'Home_and_Kitchen':
    data_all = data[['sessionID', 'itemID', 'text', 'category','review_time', 'reviewText', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6],attr_list[7],attr_list[8]]]
else:
    data_all = data[['sessionID', 'itemID', 'text', 'category','review_time', 'reviewText', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]
data_all

#session:  11817
#item:  11164


,sessionID,itemID,text,category,review_time,reviewText,Price,Quality,Functionality,Size,Protection,Material,Comfort
0,1,1,Duke Set Tool for #220 / #330 Body Traps,Hunting,1364947200,The Seller did properly describe the trap set ...,33700.0,116953.0,175226.0,230951.0,266521.0,303123.0,0.0
1,1,2,nan,nan,1392249600,"Nice fiber optic sight, and it produces a red/...",33701.0,33701.0,33701.0,33701.0,33701.0,33701.0,334954.0
2,1,3,nan,nan,1392854400,"This ammo case is nice, well thought out, I'm ...",0.0,0.0,175227.0,0.0,0.0,0.0,0.0
3,1,4,nan,nan,1393200000,For the money this belt is probably a good buy...,33702.0,116954.0,153038.0,230952.0,0.0,116954.0,0.0
4,1,5,nan,nan,1393804800,"I really like this 12 ga shotgun ammo pouch, i...",33701.0,33701.0,0.0,0.0,0.0,0.0,0.0
5,1,6,nan,nan,1397001600,"This Is A Very Neat Tactical Vest, &#34;A Best...",33703.0,116955.0,175228.0,230953.0,266522.0,303124.0,230953.0
6,1,7,nan,nan,1402876800,"It does ""Work"" as stated, and I would have giv...",0.0,0.0,175229.0,0.0,0.0,0.0,0.0
7,1,8,"VISM by NcStar Gun Case (CV2910-34), Black, 34...",Rifle Cases,1402876800,"This is a very nice case, it's size is (34x10)...",33704.0,116956.0,0.0,230954.0,266523.0,33704.0,0.0
8,1,9,nan,nan,1402876800,"Nice Sling, it seems to be heavy duty, and I w...",33705.0,0.0,0.0,230955.0,0.0,33705.0,0.0
9,2,10,Fury Tactical Boot Knife,Tactical Knives,1355788800,"This is a really well made knife, for such a l...",0.0,0.0,175231.0,0.0,0.0,0.0,0.0


In [5]:
session_path = './dict/' + meta_dataset + '/session_data.csv'
data_all.to_csv(session_path)